# **Understanding the Amazon from Space Kaggle Project**

### **Combine the images in 'test-jpg' folder with images in 'test-jpg-additional' folder**

In [4]:
import subprocess as sbp
# import os

In [7]:
path='/notebooks/Hamoye-Internship/StageD/Data/planet/test-jpg-additional'
fol = os.listdir('/notebooks/Hamoye-Internship/StageD/Data/planet/test-jpg-additional')
p2 = '/notebooks/Hamoye-Internship/StageD/Data/planet/test-jpg'

for i in fol:
    p1 = os.path.join(path,i)
    p3 = 'cp -r ' + p1 +' ' + p2+'/.'
    sbp.Popen(p3,shell=True)

#### **Import the required libraries**

In [2]:
import numpy as np
import pandas as pd
import os
# import cv2
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow.keras.layers import Dropout, Flatten, Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback, ReduceLROnPlateau
from tensorflow.keras import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.backend import clear_session


**Initialize the Hyperparameters**

In [3]:
INPUT_SHAPE = (128, 128, 3) # Image Dimensions
BATCH_SIZE = 128
DROPOUT_RATE = 0.5
EPOCHS = 24
LR = 0.0001 # Learning Rate
REG_STRENGTH = 0.01 # Regularization Strength
NFOLDS = 5 # No of folds for cross validation
WORKERS = 4 # Multithreading no of threads
MAXQ = 10 # Max Queue size for multithreading
THRES = [0.2] * 17 # Threshold for truth value of label, applied on sigmoid output

**Create the path string for the images and csv files**

In [8]:
TRAIN_PATH = '/notebooks/Hamoye-Internship/StageD/Data/planet/train-jpg'
TEST_PATH = '/notebooks/Hamoye-Internship/StageD/Data/planet/test-jpg'

TRAIN_CSV_PATH = '/notebooks/Hamoye-Internship/StageD/Data/planet/train_classes.csv'
TEST_CSV_PATH = '/notebooks/Hamoye-Internship/StageD/Data/planet/sample_submission.csv'

In [9]:
#Check the number of images in the test-jpg folder
len(os.listdir(TEST_PATH))

61191

**Construct dataframes holding training and test data**

In [10]:
df_train = pd.read_csv(TRAIN_CSV_PATH)
df_test = pd.read_csv(TEST_CSV_PATH)

df_train['image_name'] = df_train['image_name'].astype(str) + '.jpg'
df_test['image_name'] = df_test['image_name'].astype(str) + '.jpg'

df_test['tags'] = df_test['tags'].apply(lambda x: x.split(' '))

In [12]:
df_train.head()

,image_name,tags
0,train_0.jpg,haze primary
1,train_1.jpg,agriculture clear primary water
2,train_2.jpg,clear primary
3,train_3.jpg,clear primary
4,train_4.jpg,agriculture clear habitation primary road


In [13]:
df_test.head()

,image_name,tags
0,test_0.jpg,"[primary, clear, agriculture, road, water]"
1,test_1.jpg,"[primary, clear, agriculture, road, water]"
2,test_2.jpg,"[primary, clear, agriculture, road, water]"
3,test_3.jpg,"[primary, clear, agriculture, road, water]"
4,test_4.jpg,"[primary, clear, agriculture, road, water]"


In [14]:
X_train_files = np.array(df_train['image_name'].tolist())
X_train_files.reshape((X_train_files.shape[0], 1))

y_train = np.array(df_train['tags'].tolist())

**Create a list of the unique labels only**

In [15]:
labels = []

for tag in df_train['tags'].values:
    labels_in_tag = tag.split(' ')
    for label in labels_in_tag:
        if label not in labels:
            labels.append(label)
        
labels.sort()
print(labels)

['agriculture', 'artisinal_mine', 'bare_ground', 'blooming', 'blow_down', 'clear', 'cloudy', 'conventional_mine', 'cultivation', 'habitation', 'haze', 'partly_cloudy', 'primary', 'road', 'selective_logging', 'slash_burn', 'water']


**Create a CNN model with the VGG architecture**

In [16]:
def create_model():
    model = Sequential()
    model.add(InputLayer(INPUT_SHAPE))
    model.add(VGG16(weights='imagenet', include_top=False))
    model.add(Flatten())
    model.add(Dense(17, activation='sigmoid'))
    return model

In [17]:
clear_session()

In [18]:
model = create_model()
model.summary()

2022-09-19 13:16:33.387736: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 13:16:33.433760: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 13:16:33.434154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 13:16:33.436208: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 13:16:33.436573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read f

58900480/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 17)                139281    
                                                                 
Total params: 14,853,969
Trainable params: 14,853,969
Non-trainable params: 0
_________________________________________________________________


**Callback function to measure the F2 score of the validation set after every epoch and save the best model accordingly.**

In [19]:
def f2_score(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 5 * precision * recall / (4 * precision + recall)
    f_score = tf.where(tf.math.is_nan(f_score), tf.zeros_like(f_score), f_score)
    
    return tf.reduce_mean(f_score)

In [20]:
num_fold = 0

y_test = []

folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=1).split(X_train_files, y_train)

**Iterate through each fold and calculate the F2 scores of the validation set after each epoch**

**Save the best F2 scores model for each cross validation iteration.**

In [22]:
for train_index, val_index in folds:
    X_train_files_fold = X_train_files[train_index]
    y_train_fold = y_train[train_index]
    X_val_files_fold = X_train_files[val_index]
    y_val_fold = np.array(y_train[val_index])
    
    train_df = pd.DataFrame(list(zip(X_train_files_fold, y_train_fold)), columns = ['image_name', 'tags'])
    val_df = pd.DataFrame(list(zip(X_val_files_fold, y_val_fold)), columns = ['image_name', 'tags'])
    
    train_df['tags'] = train_df['tags'].apply(lambda x: x.split(' '))
    val_df['tags'] = val_df['tags'].apply(lambda x: x.split(' '))

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True
    )
    
    train_generator = train_datagen.flow_from_dataframe(
        train_df,
        directory=TRAIN_PATH,
        x_col='image_name',
        y_col='tags',
        target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        classes=labels,
    )
    
    val_datagen = ImageDataGenerator(
        rescale=1./255
    )
    
    val_generator = val_datagen.flow_from_dataframe(
        val_df,
        directory=TRAIN_PATH,
        x_col='image_name',
        y_col='tags',
        target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        classes=labels,
    )
    
    test_datagen = ImageDataGenerator(
        rescale=1./255
    )
    
    test_generator = test_datagen.flow_from_dataframe(
        df_test,
        directory=TEST_PATH,
        x_col='image_name',
        y_col='tags',
        target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        classes=labels,
        shuffle=False,
    )

    model_path_of_fold = os.path.join('', 'weights_of_fold_' + str(num_fold) + '.h5')
    
    clear_session()
    model = create_model()
    
    adam = Adam(learning_rate=LR)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[f2_score])
    
    callbacks = [
        ModelCheckpoint(model_path_of_fold, monitor='val_f2_score', save_best_only=True, mode='max'),
        ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, mode='min', min_lr=0.000001)
    ]
    
    model.fit_generator(train_generator, epochs=EPOCHS, validation_data=val_generator, callbacks=callbacks,
                       workers=WORKERS, use_multiprocessing=True, max_queue_size=MAXQ)

    model.load_weights(model_path_of_fold)

    p_test = model.predict_generator(test_generator, workers=WORKERS, use_multiprocessing=True, max_queue_size=MAXQ)
    y_test.append(p_test)

Found 32383 validated image filenames belonging to 17 classes.
Found 8096 validated image filenames belonging to 17 classes.
Found 61191 validated image filenames belonging to 17 classes.


/tmp/ipykernel_342/2317630506.py:76: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=EPOCHS, validation_data=val_generator, callbacks=callbacks,


Epoch 1/24


2022-09-19 13:22:39.212187: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


253/253 [==============================] - 159s 588ms/step - loss: 0.1526 - f2_score: 0.8073 - val_loss: 0.1231 - val_f2_score: 0.8602 - lr: 1.0000e-04
Epoch 2/24
253/253 [==============================] - 109s 427ms/step - loss: 0.1168 - f2_score: 0.8618 - val_loss: 0.1080 - val_f2_score: 0.8797 - lr: 1.0000e-04
Epoch 3/24
253/253 [==============================] - 106s 414ms/step - loss: 0.1077 - f2_score: 0.8715 - val_loss: 0.1050 - val_f2_score: 0.8802 - lr: 1.0000e-04
Epoch 4/24
253/253 [==============================] - 106s 414ms/step - loss: 0.1042 - f2_score: 0.8775 - val_loss: 0.1000 - val_f2_score: 0.8907 - lr: 1.0000e-04
Epoch 5/24
253/253 [==============================] - 106s 415ms/step - loss: 0.1012 - f2_score: 0.8821 - val_loss: 0.1015 - val_f2_score: 0.8920 - lr: 1.0000e-04
Epoch 6/24
253/253 [==============================] - 106s 414ms/step - loss: 0.0987 - f2_score: 0.8853 - val_loss: 0.0952 - val_f2_score: 0.8978 - lr: 1.0000e-04
Epoch 7/24
253/253 [=============

...

/tmp/ipykernel_342/2317630506.py:81: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  p_test = model.predict_generator(test_generator, workers=WORKERS, use_multiprocessing=True, max_queue_size=MAXQ)


Found 32384 validated image filenames belonging to 17 classes.
Found 8095 validated image filenames belonging to 17 classes.
Found 61191 validated image filenames belonging to 17 classes.


/tmp/ipykernel_342/2317630506.py:76: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=EPOCHS, validation_data=val_generator, callbacks=callbacks,


Epoch 1/24
253/253 [==============================] - 141s 544ms/step - loss: 0.1540 - f2_score: 0.8050 - val_loss: 0.1250 - val_f2_score: 0.8611 - lr: 1.0000e-04
Epoch 2/24
253/253 [==============================] - 107s 418ms/step - loss: 0.1161 - f2_score: 0.8627 - val_loss: 0.1062 - val_f2_score: 0.8838 - lr: 1.0000e-04
Epoch 3/24
253/253 [==============================] - 108s 423ms/step - loss: 0.1085 - f2_score: 0.8709 - val_loss: 0.1015 - val_f2_score: 0.8904 - lr: 1.0000e-04
Epoch 4/24
253/253 [==============================] - 107s 416ms/step - loss: 0.1040 - f2_score: 0.8772 - val_loss: 0.1003 - val_f2_score: 0.8883 - lr: 1.0000e-04
Epoch 5/24
253/253 [==============================] - 108s 421ms/step - loss: 0.1021 - f2_score: 0.8801 - val_loss: 0.0952 - val_f2_score: 0.8931 - lr: 1.0000e-04
Epoch 6/24
253/253 [==============================] - 107s 417ms/step - loss: 0.0997 - f2_score: 0.8822 - val_loss: 0.0967 - val_f2_score: 0.8928 - lr: 1.0000e-04
Epoch 7/24
253/253 [==

/tmp/ipykernel_342/2317630506.py:81: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  p_test = model.predict_generator(test_generator, workers=WORKERS, use_multiprocessing=True, max_queue_size=MAXQ)


**Take average of all predictions (OOF) generated during each fold of validation on the test set.
Attach predictions to the test dataframe**

In [23]:
result = np.array(y_test[0])
for i in range(1, NFOLDS):
    result += np.array(y_test[i])
result /= NFOLDS
result = pd.DataFrame(result, columns = labels)
result.head()

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0.013791,0.000002,0.000083,0.029518,0.000438,0.995827,4.272908e-07,5.318260e-07,0.004644,0.001454,0.001496,0.002401,0.999996,0.002821,0.003945,0.000043,0.005010
1,0.034931,0.000010,0.000436,0.012601,0.014450,0.995650,1.806414e-07,2.356175e-06,0.028148,0.001482,0.000116,0.004561,0.999997,0.001493,0.002478,0.000297,0.003139
2,0.024923,0.000014,0.000315,0.000250,0.000397,0.000080,1.971538e-05,1.144057e-05,0.007933,0.000959,0.000037,0.999811,0.999986,0.005967,0.000459,0.000086,0.039902
3,0.773282,0.000086,0.001505,0.003298,0.000528,0.934364,2.253891e-07,9.604149e-06,0.669514,0.004467,0.000945,0.077756,0.999998,0.011414,0.001094,0.003309,0.044147
4,0.015437,0.000006,0.000134,0.000005,0.000008,0.000673,1.505876e-01,7.611218e-07,0.004513,0.000794,0.000831,0.852313,0.846871,0.004713,0.000012,0.000007,0.025278


**Construct the csv file of predictions on test set, convert the binary labels to their respective labels.**

In [25]:
preds = []
for i in range(result.shape[0]):
    a = result.iloc[[i]]
    a = a.apply(lambda x: x > THRES, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))
    
df_test['tags'] = preds
df_test['image_name'] = df_test['image_name'].astype(str).str.slice(stop=-4)
df_test.to_csv('submit.csv', index=False)

### **Summary of Work Done**

- This is a multilabel classification problem. Hence, classes are encoded with binary values, each class gets a column containing truth value depicting if the image belongs to the label or not.

- The dataset comprises of Images. Images have a large no of features, deep learning is particularly suited for these kinds of tasks.

- Since images have a lot of local information, we can make do with lesser number of weights than a fully connected layer by using a CNN.

- There is a huge class imbalance. Some labels occur on more than 37512 images, while some occur on as low as 99.
Hence, I did not make a validation and train split, because some classes/combinations in that case may never appear in the training data due to randomness of split.

- Instead, I used cross validation, and Out Of Fold(OOF) Approach. For every iteration of cross validation, I opbtained the sigmoid values on the test set and then averaged them all.
Also, I used sigmoid activation in the output layer because softmax gives probabilities and is not used in multilabel classification problems.